# Extração de dados para o projeto

## Extraindo todas empresas do Brasil da categoria BESST

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: CONFIGURAÇÃO DOS SETORES-ALVO ---
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final_com_tipo(setores):
    print("Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        tipo_ativo = stock.get('type')
        if setor_brapi in setores and tipo_ativo in ['stock', 'fund']:
            ticker = f"{stock.get('stock')}.SA"
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo'),
                'tipo': tipo_ativo 
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos dos tipos 'stock' e 'fund' serão analisados.")

    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")
            stock = yf.Ticker(ticker)
            info = stock.info
            
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Tipo": brapi_data['tipo'],
                "Market Cap": info.get("marketCap"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final_com_tipo(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Tipo', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Final (Ações e Fundos) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\finance-manager\data"
        output_filename = "scanner_acoes_e_fundos_filtrado.csv"
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_csv(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo CSV: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

## Extraindo dividendos

In [ ]:
import os
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Carrega variáveis do .env
load_dotenv()

# Caminho do arquivo CSV com os tickers
csv_path = os.getenv("FILTERED_COMPANIES_PATH")

# Lê o CSV e extrai os tickers
df = pd.read_csv(csv_path)
tickers = df['Ticker'].dropna().unique().tolist()

# Define o período de 7 anos
end_date = datetime.today()
start_date = end_date - timedelta(days=7*365)

todos_dividendos = []

for ticker in tickers:
    try:
        acao = yf.Ticker(ticker)
        dividendos = acao.dividends

        # Remove timezone do índice
        dividendos.index = dividendos.index.tz_localize(None)

        # Filtra dividendos dos últimos 7 anos
        dividendos = dividendos[dividendos.index >= start_date]

        if not dividendos.empty:
            df_div = dividendos.reset_index()
            df_div.columns = ['Data', 'Valor']
            df_div['Ticker'] = ticker.replace('.SA', '')
            todos_dividendos.append(df_div)

            print(f"✅ Dividendos coletados para {ticker}")
        else:
            print(f"⚠️ Sem dividendos recentes para {ticker}")
    except Exception as e:
        print(f"❌ Erro ao processar {ticker}: {e}")

# Junta tudo em um único DataFrame
df_final = pd.concat(todos_dividendos, ignore_index=True)

# Salva em um único CSV
df_final.to_csv("../data/todos_dividendos.csv", index=False)

print("🏁 Finalizado! Dividendos salvos em todos_dividendos.csv")


### Dividendo por ano

In [ ]:
import pandas as pd

# Ler o arquivo CSV
df = pd.read_csv('../data/todos_dividendos.csv')

# Converter a coluna 'Data' para datetime e extrair o ano
df['Data'] = pd.to_datetime(df['Data'])
df['ano'] = df['Data'].dt.year

# Renomear colunas para minúsculas
df = df.rename(columns={'Ticker': 'ticket', 'Valor': 'dividendo', 'Data': 'data'})

# Somar os dividendos por ano e ticket
soma_por_ano_ticket = df.groupby(['ano', 'ticket'])['dividendo'].sum().reset_index()

# Salvar o resultado em um CSV
soma_por_ano_ticket.to_csv('../data/dividendos_ano.csv', index=False)
print("Arquivo 'dividendos_ano.csv' gerado com sucesso!")

In [5]:
import pandas as pd 

div_ano = pd.read_csv('../data/dividendos_ano.csv')
print(div_ano.head())

    ano  ticket  dividendo
0  2018   AFLT3   0.104383
1  2018   AGRO3   0.762051
2  2018  ALUP11   0.160019
3  2018   ALUP3   0.053340
4  2018   ALUP4   0.053340


### Dividendos por ano resumido (DY5 e DY12M)

In [ ]:
import pandas as pd

# Carregar os dividendos por ano e ticket
df = pd.read_csv('../data/dividendos_ano.csv')

# Descobrir o último ano disponível
ultimo_ano = df['ano'].max()

# Dividendos dos últimos 5 anos
div_5anos = df[df['ano'] >= ultimo_ano - 4]
soma_5anos = div_5anos.groupby('ticket')['dividendo'].sum().reset_index()
soma_5anos = soma_5anos.rename(columns={'dividendo': 'valor_5anos'})

# Dividendos dos últimos 12 meses
div_12m = df[df['ano'] == ultimo_ano]
soma_12m = div_12m[['ticket', 'dividendo']].rename(columns={'dividendo': 'valor_12m'})

# Juntar os dois
resumo = pd.merge(soma_5anos, soma_12m, on='ticket', how='outer').fillna(0)

# Reorganizar colunas
resumo = resumo[['ticket', 'valor_5anos', 'valor_12m']]

# Salvar resultado
resumo.to_csv('../data/dividendos_ano_resumo.csv', index=False)
print("Arquivo 'dividendos_ano_resumo.csv' gerado com sucesso!")
print(resumo.head())
print(resumo.shape)


Arquivo 'dividendos_ano_resumo.csv' gerado com sucesso!
   ticket  valor_5anos  valor_12m
0   ABCB4     6.902438   1.086000
1   AFLT3     1.894183   0.017041
2   AGRO3    12.644483   0.000000
3   ALOS3     3.347996   0.691226
4  ALUP11     5.310194   0.450000


## Extraindo valores das ações

In [ ]:
## Rever o codigo abaixo

## Script de Transformação

In [ ]:
# transform.py (Versão com Cálculo de DY Padronizado)
import yfinance as yf
import pandas as pd
import os
from datetime import datetime

def classify_stock_profile(price: float, market_cap: float) -> str:
    """
    Classifica a ação com base no Market Cap e Preço.
    """
    if price is not None and price < 1.0:
        return "Penny Stock"
    if market_cap is None or market_cap == 0:
        return "N/A"
    if market_cap > 50_000_000_000:
        return "Blue Chip"
    if market_cap > 10_000_000_000:
        return "Mid Cap"
    if market_cap > 2_000_000_000:
        return "Small Cap"
    return "Micro Cap"

def get_market_sentiment_and_details(ticker_obj: yf.Ticker) -> dict:
    """
    Usa 'recommendations_summary' para maior robustez.
    """
    sentiment_data = {
        'Sentimento Gauge': 50.0, 'Strong Buy': 0, 'Buy': 0,
        'Hold': 0, 'Sell': 0, 'Strong Sell': 0
    }
    try:
        summary = ticker_obj.recommendations_summary
        if summary is None or summary.empty:
            return sentiment_data
        rec_counts = summary.iloc[-1]
        strong_buy = int(rec_counts.get('strongBuy', 0))
        buy = int(rec_counts.get('buy', 0))
        hold = int(rec_counts.get('hold', 0))
        sell = int(rec_counts.get('sell', 0))
        strong_sell = int(rec_counts.get('strongSell', 0))
        sentiment_data.update({
            'Strong Buy': strong_buy, 'Buy': buy, 'Hold': hold,
            'Sell': sell, 'Strong Sell': strong_sell
        })
        weighted_score = (strong_buy * 2) + (buy * 1) + (hold * 0) + (sell * -1) + (strong_sell * -2)
        total_recommendations = strong_buy + buy + hold + sell + strong_sell
        if total_recommendations == 0:
            return sentiment_data
        avg_score = weighted_score / total_recommendations
        normalized_sentiment = ((avg_score + 2) / 4) * 100
        sentiment_data['Sentimento Gauge'] = normalized_sentiment
        return sentiment_data
    except Exception as e:
        print(f" (Aviso: Falha ao buscar sentimento - {e}) ", end="")
        return sentiment_data

def fetch_stock_data(ticker: str, info_original: pd.Series) -> dict:
    """
    Busca e processa os dados de um único ticker, com lógica de fallback para Market Cap e DY 5 anos.
    """
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="5y")

    market_cap = info.get('marketCap')
    current_price = info.get("currentPrice", 0.0)
    if not market_cap or market_cap == 0:
        shares_outstanding = info.get('sharesOutstanding', 0)
        if shares_outstanding > 0 and current_price > 0:
            market_cap = shares_outstanding * current_price
    if not market_cap or market_cap == 0:
        market_cap = info_original.get('Market Cap', 0)

    # --- Cálculos de Indicadores ---
    # Dividend Yield 12 meses (já em percentual, apenas valida)
    dy_12m = info.get("trailingAnnualDividendYield", 0.0) * 100 if info.get("trailingAnnualDividendYield") is not None else 0.0
    if dy_12m > 50 or dy_12m < 0:
        dy_12m = 0.0  # Proteção contra valores absurdos

    # Dividend Yield 5 anos (evitar multiplicação por 100 e adicionar fallback)
    dy_5y = info.get("fiveYearAvgDividendYield", 0.0) if info.get("fiveYearAvgDividendYield") is not None else 0.0
    # Validação: valores fora de 0-50% são considerados inválidos
    if dy_5y > 50 or dy_5y < 0:
        print(f" (Aviso: DY 5 anos de {ticker} ({dy_5y:.2f}%) inválido, calculando manualmente...) ", end="")
        dy_5y = 0.0
        try:
            dividends = stock.dividends
            if not dividends.empty:
                # Soma os dividendos dos últimos 5 anos
                dividends_5y = dividends[dividends.index >= (datetime.now() - pd.Timedelta(days=5*365))]
                total_dividends = dividends_5y.sum()
                # Calcula preço médio dos últimos 5 anos
                avg_price = history['Close'].mean() if not history.empty else current_price
                if avg_price > 0:
                    dy_5y = (total_dividends / 5) / avg_price * 100  # Média anual de dividendos / preço médio
        except Exception as e:
            print(f" (Aviso: Falha ao calcular DY 5 anos manualmente para {ticker} - {e}) ", end="")

    growth_price = 0.0
    if not history.empty and len(history['Close']) > 1 and history['Close'].iloc[0] > 0:
        growth_price = ((history['Close'].iloc[-1] / history['Close'].iloc[0]) - 1) * 100
    payout_ratio = info.get("payoutRatio", 0.0) * 100 if info.get("payoutRatio") is not None else 0.0
    sentiment_info = get_market_sentiment_and_details(stock)

    dados = {
        "Empresa": info_original.get('Empresa', 'N/A'),
        "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'),
        "Tipo": info_original.get('Tipo', 'N/A'),
        "Market Cap": market_cap,
        "Logo": info_original.get('Logo', 'N/A'),
        "Preço Atual": current_price,
        "P/L": info.get("trailingPE"),
        "P/VP": info.get("priceToBook"),
        "DY (Taxa 12m, %)": dy_12m,
        "DY 5 Anos Média (%)": dy_5y,
        "Último Dividendo": info.get("lastDividendValue"),
        "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s', errors='coerce'),
        "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s', errors='coerce'),
        "Payout Ratio (%)": payout_ratio,
        "Crescimento Preço (%)": growth_price,
        "ROE (%)": info.get("returnOnEquity", 0.0) * 100 if info.get("returnOnEquity") is not None else 0.0,
        "Dívida Total": info.get('totalDebt'),
        "EBITDA": info.get('ebitda'),
        "Perfil da Ação": classify_stock_profile(current_price, market_cap)
    }
    dados.update(sentiment_info)
    if dados.get("EBITDA") and dados["EBITDA"] != 0 and dados.get("Dívida Total"):
        dados["Dívida/EBITDA"] = dados["Dívida Total"] / dados["EBITDA"]
    else:
        dados["Dívida/EBITDA"] = None
    return dados

def main():
    arquivo_input = r"E:\Github\Unicamp\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv"
    output_path = r"E:\Github\Unicamp\finance-manager\data\relatorio_analise_b3.csv"
    try:
        df_input = pd.read_csv(arquivo_input)
        tickers_para_analisar = df_input['Ticker'].tolist()
    except FileNotFoundError:
        print(f"❌ ERRO CRÍTICO: Arquivo de entrada '{arquivo_input}' não encontrado.")
        return
    dados_detalhados = {}
    erros = []
    print("\nIniciando coleta de dados via yfinance...")
    for ticker in tickers_para_analisar:
        print(f"Processando: {ticker}...", end="")
        try:
            info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
            dados_detalhados[ticker] = fetch_stock_data(ticker, info_original)
            print(" ✅")
        except Exception as e:
            erros.append(ticker)
            print(f" ❌ (Erro: {e})")
    if not dados_detalhados:
        print("\n❌ NENHUM DADO FOI COLETADO COM SUCESSO. O arquivo CSV não será gerado.")
        return
    df_resultado = pd.DataFrame.from_dict(dados_detalhados, orient="index")
    df_resultado.fillna(0, inplace=True)
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_resultado.to_csv(output_path)
        print(f"\n✅ SUCESSO! Relatório salvo com sucesso em: {output_path}")
    except Exception as e:
        print(f"❌ ERRO INESPERADO AO SALVAR O ARQUIVO: {e}")

if __name__ == "__main__":
    main()


In [ ]:
    # arquivo_input = r"C:\Users\01701805\Desktop\Projetos GL\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv"
    # output_path = r"C:\Users\01701805\Desktop\Projetos GL\finance-manager\data\relatorio_analise_b3.csv"

In [ ]:
# app.py:

# df = load_data(r"C:\Users\01701805\Desktop\Projetos GL\finance-manager\data\relatorio_analise_b3.csv")


In [ ]:
# import pandas as pd

# df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\relatorio_completo_b3.xlsx')

# print(df_dados.head(2))